<a href="https://colab.research.google.com/github/scott-cjx/1015-Project/blob/main/1015_WeatherTaxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
def get_taxi_avail():
  url = "https://api.data.gov.sg/v1/transport/taxi-availability"

  # Send request and get response
  response = requests.get(url)

  if response.status_code != 200:
    return

  # Parse JSON data
  data = response.json()

  data_coordinates = data['features'][0]['geometry']['coordinates']
  df_coordinates = pd.DataFrame(data_coordinates)
  df_coordinates.columns = ["LONGITUDE", "LATITUDE"]

  ts = data['features'][0]['properties']['timestamp']
  dt_object = datetime.fromisoformat(ts)

  ts_date = dt_object.date()
  ts_time = dt_object.time()
  df_coordinates["DATE"] = ts_date
  df_coordinates["TIME"] = ts_time

  return df_coordinates

# taxi_avail = get_taxi_avail()

In [ ]:
def get_area_meta():
  url = 'https://api.data.gov.sg/v1/environment/2-hour-weather-forecast'

  # Send request and get response
  response = requests.get(url)

  if response.status_code != 200:
    return

  # Parse JSON data
  data = response.json()
  data_areameta = data['area_metadata']
  df = pd.DataFrame(data_areameta)
  latitude_values = df['label_location'].apply(lambda x: x['latitude'])
  df['LATITUDE'] = latitude_values
  longitude_values = df['label_location'].apply(lambda x: x['longitude'])
  df['LONGITUDE'] = longitude_values
  df = df.drop(columns=['label_location'])

  return df

# get_area_meta()

In [ ]:
def get_weather():
  url = 'https://api.data.gov.sg/v1/environment/2-hour-weather-forecast'

  # Send request and get response
  response = requests.get(url)

  if response.status_code != 200:
    return

  # Parse JSON data
  data = response.json()

  data_weather = data['items'][0]['forecasts']
  df = pd.DataFrame(data_weather)
  df.columns = ["AREA", "FORECAST"]

  ts = data['items'][0]['timestamp']
  df['timestamp'] = ts
  upd_ts = data['items'][0]['update_timestamp']
  df['update_timestamp'] = upd_ts
  valid_start_ts = data['items'][0]['valid_period']['start']
  df['valid_start'] = valid_start_ts
  valid_end_ts = data['items'][0]['valid_period']['end']
  df['valid_end'] = valid_end_ts

  return df

# get_weather()

In [ ]:
import os

def append_pkl(pkl_fp, to_add_df):
  try:
    df = pd.read_pickle(pkl_fp)
  except FileNotFoundError:
    df = pd.DataFrame()

  df = pd.concat([df, to_add_df], ignore_index=True)
  df.to_pickle(pkl_fp)

def read_pkl(pkl_fp):
  df = pd.read_pickle(pkl_fp)
  return df


In [ ]:
# append_pkl("taxi_log.pkl", get_taxi_avail())

In [ ]:
read_pkl("taxi_log.pkl").head()

In [ ]:
# append_pkl("weather_log.pkl", get_weather())

In [ ]:
weather_df = read_pkl("weather_log.pkl")

In [ ]:
print("f")

Clean both datasets


Function to convert coordinates to x y


Classify to area zones (find closest zone and classify)

In [ ]:
!pip install haversine

In [ ]:
import haversine as hs;
def get_zone(longitude, latitude):
  position = (latitude, longitude)

  area_metadata = [
    {
      "name": "Ang Mo Kio",
      "label_location": {
        "latitude": 1.375,
        "longitude": 103.839
      }
    },
    {
      "name": "Bedok",
      "label_location": {
        "latitude": 1.321,
        "longitude": 103.924
      }
    },
    {
      "name": "Bishan",
      "label_location": {
        "latitude": 1.350772,
        "longitude": 103.839
      }
    },
    {
      "name": "Boon Lay",
      "label_location": {
        "latitude": 1.304,
        "longitude": 103.701
      }
    },
    {
      "name": "Bukit Batok",
      "label_location": {
        "latitude": 1.353,
        "longitude": 103.754
      }
    },
    {
      "name": "Bukit Merah",
      "label_location": {
        "latitude": 1.277,
        "longitude": 103.819
      }
    },
    {
      "name": "Bukit Panjang",
      "label_location": {
        "latitude": 1.362,
        "longitude": 103.77195
      }
    },
    {
      "name": "Bukit Timah",
      "label_location": {
        "latitude": 1.325,
        "longitude": 103.791
      }
    },
    {
      "name": "Central Water Catchment",
      "label_location": {
        "latitude": 1.38,
        "longitude": 103.805
      }
    },
    {
      "name": "Changi",
      "label_location": {
        "latitude": 1.357,
        "longitude": 103.987
      }
    },
    {
      "name": "Choa Chu Kang",
      "label_location": {
        "latitude": 1.377,
        "longitude": 103.745
      }
    },
    {
      "name": "Clementi",
      "label_location": {
        "latitude": 1.315,
        "longitude": 103.76
      }
    },
    {
      "name": "City",
      "label_location": {
        "latitude": 1.292,
        "longitude": 103.844
      }
    },
    {
      "name": "Geylang",
      "label_location": {
        "latitude": 1.318,
        "longitude": 103.884
      }
    },
    {
      "name": "Hougang",
      "label_location": {
        "latitude": 1.361218,
        "longitude": 103.886
      }
    },
    {
      "name": "Jalan Bahar",
      "label_location": {
        "latitude": 1.347,
        "longitude": 103.67
      }
    },
    {
      "name": "Jurong East",
      "label_location": {
        "latitude": 1.326,
        "longitude": 103.737
      }
    },
    {
      "name": "Jurong Island",
      "label_location": {
        "latitude": 1.266,
        "longitude": 103.699
      }
    },
    {
      "name": "Jurong West",
      "label_location": {
        "latitude": 1.34039,
        "longitude": 103.705
      }
    },
    {
      "name": "Kallang",
      "label_location": {
        "latitude": 1.312,
        "longitude": 103.862
      }
    },
    {
      "name": "Lim Chu Kang",
      "label_location": {
        "latitude": 1.423,
        "longitude": 103.717332
      }
    },
    {
      "name": "Mandai",
      "label_location": {
        "latitude": 1.419,
        "longitude": 103.812
      }
    },
    {
      "name": "Marine Parade",
      "label_location": {
        "latitude": 1.297,
        "longitude": 103.891
      }
    },
    {
      "name": "Novena",
      "label_location": {
        "latitude": 1.327,
        "longitude": 103.826
      }
    },
    {
      "name": "Pasir Ris",
      "label_location": {
        "latitude": 1.37,
        "longitude": 103.948
      }
    },
    {
      "name": "Paya Lebar",
      "label_location": {
        "latitude": 1.358,
        "longitude": 103.914
      }
    },
    {
      "name": "Pioneer",
      "label_location": {
        "latitude": 1.315,
        "longitude": 103.675
      }
    },
    {
      "name": "Pulau Tekong",
      "label_location": {
        "latitude": 1.403,
        "longitude": 104.053
      }
    },
    {
      "name": "Pulau Ubin",
      "label_location": {
        "latitude": 1.404,
        "longitude": 103.96
      }
    },
    {
      "name": "Punggol",
      "label_location": {
        "latitude": 1.401,
        "longitude": 103.904
      }
    },
    {
      "name": "Queenstown",
      "label_location": {
        "latitude": 1.291,
        "longitude": 103.78576
      }
    },
    {
      "name": "Seletar",
      "label_location": {
        "latitude": 1.404,
        "longitude": 103.869
      }
    },
    {
      "name": "Sembawang",
      "label_location": {
        "latitude": 1.445,
        "longitude": 103.818495
      }
    },
    {
      "name": "Sengkang",
      "label_location": {
        "latitude": 1.384,
        "longitude": 103.891443
      }
    },
    {
      "name": "Sentosa",
      "label_location": {
        "latitude": 1.243,
        "longitude": 103.832
      }
    },
    {
      "name": "Serangoon",
      "label_location": {
        "latitude": 1.357,
        "longitude": 103.865
      }
    },
    {
      "name": "Southern Islands",
      "label_location": {
        "latitude": 1.208,
        "longitude": 103.842
      }
    },
    {
      "name": "Sungei Kadut",
      "label_location": {
        "latitude": 1.413,
        "longitude": 103.756
      }
    },
    {
      "name": "Tampines",
      "label_location": {
        "latitude": 1.345,
        "longitude": 103.944
      }
    },
    {
      "name": "Tanglin",
      "label_location": {
        "latitude": 1.308,
        "longitude": 103.813
      }
    },
    {
      "name": "Tengah",
      "label_location": {
        "latitude": 1.374,
        "longitude": 103.715
      }
    },
    {
      "name": "Toa Payoh",
      "label_location": {
        "latitude": 1.334304,
        "longitude": 103.856327
      }
    },
    {
      "name": "Tuas",
      "label_location": {
        "latitude": 1.294947,
        "longitude": 103.635
      }
    },
    {
      "name": "Western Islands",
      "label_location": {
        "latitude": 1.205926,
        "longitude": 103.746
      }
    },
    {
      "name": "Western Water Catchment",
      "label_location": {
        "latitude": 1.405,
        "longitude": 103.689
      }
    },
    {
      "name": "Woodlands",
      "label_location": {
        "latitude": 1.432,
        "longitude": 103.786528
      }
    },
    {
      "name": "Yishun",
      "label_location": {
        "latitude": 1.418,
        "longitude": 103.839
      }
    }
  ]

  shortest_dist = {"Area": "NULL", "Distance": 6000000}

  for zone in area_metadata:
    compare = (zone['label_location']['latitude'], zone['label_location']['longitude'])
    dist = hs.haversine(compare, position)
    #See if it is the shortest distance
    if dist < shortest_dist['Distance']:
      shortest_dist['Area'] = zone['name']
      shortest_dist['Distance'] = dist

  return shortest_dist['Area']

#get_zone(103.998676,1.328944)

Add Zones to Taxi Df

In [ ]:
#Read Taxi pkl file
taxi_df = read_pkl("taxi_log.pkl")

def load_taxi_zones():
  taxi_df['Zone'] = taxi_df.apply(lambda row: get_zone(row['LONGITUDE'], row['LATITUDE']), axis=1)
  return taxi_df

#Add Locaiton Zones to dataframe
taxi_df = load_taxi_zones()

Add days of the week to taxi Df

In [ ]:
import calendar

def add_day_of_week(df):
  df['Day_of_Week'] = df['DATE'].apply(lambda x: calendar.day_name[x.weekday()])
  return df

#Add days of the week into dataframe
taxi_df = add_day_of_week(taxi_df)
taxi_df.head()
taxi_df.groupby('Day_of_Week').size()

Display Taxi Location on Matplot based on the day of the week

In [ ]:
from ipywidgets import interact

import matplotlib.patches as mpatches

#Plot all Taxis using a scatter plot based on their cords, colour sperated by Zones.
def plot_data(day_of_week):
    if(day_of_week=='All'):
        df_day = taxi_df
    else:
        # Filter data for the selected day of the week
        df_day = taxi_df[taxi_df['Day_of_Week'] == day_of_week]

    # Plot the data
    plt.figure(figsize=(150, 75))
    plt.scatter(df_day["LONGITUDE"], df_day["LATITUDE"], c=df_day["Zone"].astype('category').cat.codes)

    # Create a legend
    zone_names = df_day["Zone"].unique()
    color_map = plt.cm.get_cmap('viridis', len(zone_names))
    legend_elements = [mpatches.Patch(color=color_map(i), label=zone) for i, zone in enumerate(zone_names)]
    plt.legend(handles=legend_elements, loc='upper right',fontsize=50, title='Zone', title_fontsize='50',markerscale=10)

    plt.title(f'Taxi Data for {day_of_week}')
    plt.xlabel('Longitude',fontsize=50)
    plt.ylabel('Latitude',fontsize=50)
    plt.title(label=f'Number of Taxis: {len(df_day)}', fontsize=50)
    plt.show()

# Create a slider for day of the week
days_of_week = ['All','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
interact(plot_data, day_of_week=days_of_week)

Weather File clean-up

In [ ]:
weather_df = read_pkl("weather_log.pkl")

def clean_weather(df):
    # Convert the 'timestamp' column to datetime
    df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
    df['valid_start'] = pd.to_datetime(weather_df['valid_start'])
    df['valid_end'] = pd.to_datetime(weather_df['valid_end'])

    # Create new columns 'valid_start_date', 'valid_start_time', 'valid_end_date' and 'valid_end_time' by splitting the 'valid_start' and 'valid_end' columns
    df['weather_date'] = df['valid_start'].dt.date
    df['weather_start_time'] = df['valid_start'].dt.time
    df['weather_end_time'] = df['valid_end'].dt.time

    # Assuming 'column_to_remove' is the name of the column you want to remove
    df = df.drop('valid_start', axis=1)
    df = df.drop('valid_end', axis=1)
    df = df.drop('update_timestamp', axis=1)
    # Remove duplicates
    df = df.drop_duplicates()
    # Filter rows where hour is divisible by 2 and minute is 0
    df = df[df['weather_start_time'].apply(lambda x: x.hour % 2 == 0 and x.minute == 0)]
    return df